In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from PIL import Image
from torch.utils.data import WeightedRandomSampler
import torch.optim as optim

In [6]:
# Ścieżka do folderu z oryginalnymi obrazami

def save_to_list():
    output_root = "output"
    X = []
    y = []
    label_to_index = {label: idx for idx, label in enumerate(set(['AVM', 'Normal', 'Ulcer']))} 
    # Przechodzimy rekurencyjnie przez wszystkie pliki w folderze archive
    for root, _, files in os.walk(output_root):
        for file in files:
            if file.endswith(".bmp"):  # Obsługujemy tylko pliki BMP
                input_path = os.path.join(root, file)

                base, ext = os.path.splitext(file)
                new_filename = f"{base}{ext}"
                image = cv2.imread(input_path)
                # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                #gray zostało tymczasowo zakomentowane aby sprawdzić accuracy
                
                # zapisanie obrazu w grayscale (3x mniej danych)
                # TODO przezkalowanie z [0,255] do [0,1]
                X.append(image)
                y.append(label_to_index[new_filename.split("_")[0]])
    return X, y


# data_all = save_to_list()
# print(np.shape(data_all))
X, y = save_to_list()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=123, stratify=y_test)

## Dalsze przygotowanie danych (przetwarzanie i augmentacja)
Ponieważ ostatni kamień milowy zakończyliśmy z ilością około 3000 zdjęć o rozmiarach 512 x 512 px powinniśmy postarać się zwiększyć ilość zdjęć oraz zmiejszyć ich rozmiar (i dosłownie do 224 x 224 px (standard w DL), i w pamięci)

In [10]:
# https://discuss.pytorch.org/t/balanced-sampling-between-classes-with-torchvision-dataloader/2703/2

class ImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform # przygotowanie do zwiększenia ilości zdjęć
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])  
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

### Definiowanie transforamtorów i przypisanie do datasetów PyTorcha
Ponieważ do tej pory używaliśmy sklearn a potrzebujemy pyTorcha (jest lepszy do wybranego przez nas algorytmu uczenia) to musimy przekonwertować dane na format który PyTorch będzie rozumiał

In [67]:
train_transform = transforms.Compose([#TODO to jest placeholder 
    transforms.Resize((224, 224)),          # rozmiar do potencjalnej zmiany
    transforms.RandomRotation((-180,180)),  
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])  # do testów - bez zmian


train_dataset = ImageDataset(X_train, y_train, transform=train_transform)
test_dataset = ImageDataset(X_test, y_test, transform=test_transform)

class_counts = np.bincount(y)  
class_weights = 1.0 / class_counts 

sample_weights_tr = [class_weights[label] for label in y_train]
sample_weights_ts = [class_weights[label] for label in y_test]

samplertr = WeightedRandomSampler(sample_weights_tr, num_samples= len(sample_weights_tr), replacement=True)
samplerts = WeightedRandomSampler(sample_weights_ts, num_samples= len(sample_weights_ts), replacement=True)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler = samplertr)
test_loader = DataLoader(test_dataset, batch_size=batch_size, sampler = samplerts)

In [68]:
images, labels = next(iter(train_loader))
print(labels)

tensor([1, 2, 2, 1, 1, 0, 2, 0, 0, 2, 2, 2, 1, 2, 0, 2, 2, 1, 2, 0, 0, 0, 2, 0,
        0, 2, 1, 2, 0, 2, 0, 2])


### Wybór modelu
Wstępnie wybraliśmy algorytm CNN (Convolutional Neural Networks) ponieważ:
- Ilość zdjęć po transformacjach będzie wynosiła dziesiątki tysięcy co sprawia, że użycie tego algorytmu możliwe (ale idealnie byłoby aby miec diesiątki tysięcy oryginalnych zdjęć)
- Model powinien umieć rozpoznawać fragmenty zdjęcia do czego nadają się CNNy (robią to automatycznie)
- Jest najlepszym algorytmem do klasyfikacji obrazów (w naszej opinii)

jeśli starczy czasu spróbujemy stworzyć też inne modele aby porównać celność rożnych algorytmów.
- Transfer Learning - może być bardzo użyteczny ponieważ bez transformacji mamy 3000 tylko zdjęć (może byc overfitting przy CNN)
#### Powody przeciwko wybraniu pozotałych algorytmów:
- SVM - zbyt wolny przy dużej ilości danych
- Gradient Boosting - Nadaje się do danych tabelarycznych  - nie obrazów

In [80]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=3):
        super(CNNModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), 
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),  # ewentualnie dodać jeszcze jedną warstwę pośrednią
            nn.ReLU(),
            nn.Dropout(0.3), # zapobieganie overfittingowi do wyregulowania
            nn.Linear(512, 512),  # ewentualnie dodać jeszcze jedną warstwę pośrednią
            nn.ReLU(),
            nn.Dropout(0.3), # zapobieganie overfittingowi do wyregulowania
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


#### Wyjaśnienie kodu

self.conv_layers - zamienia tensor 224×224×3 na tensor 28×28×128, który zawiera ekstraktowane cechy obrazu.

fc_layers - warstwy klasyfikacyjne (ta część która jest okładką deepLearningu) (input - nodes - output)
(ewentualna możliwość dodania dodatkowej warstwy nodes w razie wymagań)

In [57]:
def train_and_evaluate(model, train_loader, test_loader, epochs=10, learning_rate=0.001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")
    
    # Evaluate on test set
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_acc = 100 * correct / total
    print(f"Test Accuracy: {test_acc:.2f}%")
    return model


In [79]:
model = CNNModel(num_classes=3)
trained_model = train_and_evaluate(model, train_loader, test_loader, epochs=15)

Epoch 1/15, Loss: 3.0826, Train Accuracy: 45.47%
Epoch 2/15, Loss: 0.9284, Train Accuracy: 54.53%
Epoch 3/15, Loss: 0.9020, Train Accuracy: 59.92%
Epoch 4/15, Loss: 0.8529, Train Accuracy: 62.14%
Epoch 5/15, Loss: 0.8432, Train Accuracy: 60.01%
Epoch 6/15, Loss: 0.8259, Train Accuracy: 62.40%
Epoch 7/15, Loss: 0.8172, Train Accuracy: 62.84%
Epoch 8/15, Loss: 0.8143, Train Accuracy: 64.27%
Epoch 9/15, Loss: 0.7789, Train Accuracy: 65.45%
Epoch 10/15, Loss: 0.7570, Train Accuracy: 67.49%
Epoch 11/15, Loss: 0.7727, Train Accuracy: 66.32%
Epoch 12/15, Loss: 0.7768, Train Accuracy: 66.19%
Epoch 13/15, Loss: 0.7588, Train Accuracy: 66.62%
Epoch 14/15, Loss: 0.7109, Train Accuracy: 69.97%
Epoch 15/15, Loss: 0.7302, Train Accuracy: 69.10%
Test Accuracy: 75.86%
